# Relatório de Análise de Dados Steam - Fase II

**Disciplina:** Programação para Dados  
**Aluno:** Níccolas Fernando Cassel 
**Data:** 01/09/2025

---

## 1. Introdução

Este notebook apresenta a Fase II do projeto final da disciplina de Programação para Dados, com o objetivo de analisar um conjunto de dados de jogos da plataforma Steam. Utilizando o ecossistema de ciência de dados em Python (`pandas`, `matplotlib`, `numpy`), foram desenvolvidas consultas e visualizações para gerar insights relevantes para a Fun Corp.

A análise abrange as seguintes etapas:
*   Carregamento e pré-processamento do conjunto de dados.
*   Resposta às perguntas de negócio levantadas pela Fun Corp.
*   Geração de gráficos exploratórios para auxiliar na interpretação dos dados.
*   Formulação de uma pergunta de negócio adicional e criação de um gráfico exploratório complementar.
*   Padronização do estilo visual dos gráficos.

Todo o código-fonte que suporta este relatório está organizado em módulos Python (`data_processor.py`, `analysis_functions.py`, `chart_plotting.py`) e pode ser encontrado na pasta do projeto.

---

In [ ]:
# Importação das bibliotecas e módulos necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Importação dos módulos customizados
from data_processor import load_and_preprocess_data
from analysis_functions import (
    get_top_10_metacritic_games,
    analyze_role_playing_games_metrics,
    analyze_top_paid_game_publishers,
    check_linux_support_growth,
    analyze_price_vs_recommendations_by_genre
)
from chart_plotting import ChartGenerator

# --- Configuração de Caminhos ---
DATASET_PATH = 'data/dataset/steam_games.csv' # Caminho para o dataset completo
PLOTS_DIR = 'data/plots' # Diretório onde os gráficos serão salvos

# Garante que o diretório de plots existe
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

# Inicializa o gerador de gráficos
chart_generator = ChartGenerator(output_dir=PLOTS_DIR)

print("Bibliotecas e módulos importados com sucesso.")
print(f"Diretório para salvar gráficos: '{PLOTS_DIR}'")

In [ ]:
# Carregar e pré-processar o dataset completo
try:
    df_steam = load_and_preprocess_data(DATASET_PATH)
    print("\nPré-visualização do DataFrame:")
    display(df_steam.head()) # 'display' é uma função útil em notebooks para mostrar DataFrames formatados
    print(f"\nInformações gerais do DataFrame (número de linhas e colunas): {df_steam.shape}")
    print("\nVerificação de tipos de dados:")
    display(df_steam.info())
except FileNotFoundError as e:
    print(e)
    print("Por favor, verifique se o arquivo 'steam_games.csv' está na pasta 'data/dataset'.")
except Exception as e:
    print(f"Ocorreu um erro durante o carregamento ou pré-processamento dos dados: {e}")

---
## 2. Respostas às Perguntas da Fun Corp.

A seguir, são apresentadas as análises e respostas para as perguntas de negócio levantadas pela Fun Corp., utilizando o conjunto de dados da Steam.

### Pergunta 1: Quais são os dez jogos mais bem avaliados, de acordo com o Metacritic? No caso de notas repetidas, ordenar os jogos de acordo com suas datas de lançamento (do mais velho para o mais recente).

In [ ]:
top_10_metacritic_games = get_top_10_metacritic_games(df_steam)
print("Os dez jogos mais bem avaliados segundo o Metacritic são:")
display(top_10_metacritic_games)

**Análise:**
Esta consulta destaca os jogos com a maior aprovação da crítica, conforme refletido pelas pontuações do Metacritic. A ordenação secundária por data de lançamento em caso de empate garante uma consistência na seleção. Os jogos listados representam títulos que foram historicamente bem recebidos, o que pode indicar tendências de sucesso em termos de qualidade e impacto no mercado. Para a Fun Corp., essa informação pode guiar decisões sobre quais tipos de jogos (gênero, desenvolvedor, publisher) tendem a alcançar excelência e alta crítica, servindo como benchmark para futuros projetos.

### Pergunta 2: Para jogos de role-playing, qual o número médio e máximo de: DLCs, avaliações positivas, avaliações negativas e materiais de demonstração (número de capturas de tela e filmes, somados)?

In [ ]:
rpg_metrics = analyze_role_playing_games_metrics(df_steam)
print("Métricas para Jogos de Role-Playing (RPG):")
if "message" in rpg_metrics:
    print(rpg_metrics["message"])
else:
    for metric, values in rpg_metrics.items():
        print(f"- {metric.replace('_', ' ').title()}: Média = {values['media']:.2f}, Máximo = {values['maximo']:.0f}")

**Análise:**
A análise de jogos de Role-Playing revela características importantes deste gênero. A média e o máximo de DLCs indicam o potencial de expansão e monetização contínua desses títulos. O volume de avaliações positivas e negativas oferece um panorama da satisfação dos jogadores e áreas de possível melhoria. Já o total de materiais de demonstração (screenshots e filmes) sugere o nível de investimento em marketing visual e a complexidade que os desenvolvedores consideram necessária para apresentar seus jogos ao público. Esses dados podem ajudar a Fun Corp. a entender as expectativas do mercado para jogos de RPG e a planejar seus próprios investimentos em desenvolvimento e marketing.

### Pergunta 3: Quais são as cinco empresas que mais publicam jogos pagos na plataforma? Para tais empresas, qual o número médio e mediano de avaliações positivas de seus jogos pagos?

In [ ]:
top_paid_publishers = analyze_top_paid_game_publishers(df_steam)
print("As cinco empresas que mais publicam jogos pagos e suas métricas de avaliações positivas:")
display(top_paid_publishers)

**Análise:**
Esta consulta identifica os principais players no segmento de jogos pagos da Steam, fornecendo um panorama sobre quem domina a publicação de títulos monetizados. Ao observar a média e a mediana de avaliações positivas, podemos inferir não apenas o volume de produção, mas também a capacidade dessas editoras em lançar jogos que ressoam positivamente com a audiência. Uma alta média e mediana de avaliações positivas para um grande volume de jogos pagos sugere uma editora com um portfólio robusto e bem-sucedido. Para a Fun Corp., é uma oportunidade de identificar concorrentes-chave, potenciais parceiros ou mesmo lacunas de mercado.

### Pergunta 4: O número de jogos que suportam o sistema operacional Linux cresceu entre 2018 e 2022?

In [ ]:
linux_growth_data = check_linux_support_growth(df_steam)
print("Crescimento de jogos com suporte Linux por ano (2018-2022):")
display(linux_growth_data)

# Análise de tendência
if not linux_growth_data.empty:
    first_year_count = linux_growth_data.loc[linux_growth_data['release_year'] == 2018, 'num_linux_games'].iloc[0] if 2018 in linux_growth_data['release_year'].values else 0
    last_year_count = linux_growth_data.loc[linux_growth_data['release_year'] == 2022, 'num_linux_games'].iloc[0] if 2022 in linux_growth_data['release_year'].values else 0
    
    if last_year_count > first_year_count:
        print(f"\nConclusão: Sim, o número de jogos com suporte Linux CRESCEU entre 2018 ({first_year_count} jogos) e 2022 ({last_year_count} jogos).")
    elif last_year_count < first_year_count:
        print(f"\nConclusão: Não, o número de jogos com suporte Linux DIMINUIU entre 2018 ({first_year_count} jogos) e 2022 ({last_year_count} jogos).")
    else:
        print(f"\nConclusão: O número de jogos com suporte Linux se manteve ESTÁVEL entre 2018 e 2022, ou não houve dados significativos para determinar uma tendência clara.")
else:
    print("\nNão foi possível determinar a tendência devido à falta de dados.")

**Análise:**
A evolução do suporte a Linux é um indicador da diversificação e inclusão de plataformas no mercado de jogos. Um crescimento no número de jogos com suporte a Linux entre 2018 e 2022 sinaliza uma possível expansão desse nicho de mercado, impulsionado talvez por iniciativas como o Steam Deck e a crescente popularidade de distribuições Linux. Para a Fun Corp., essa tendência pode representar uma oportunidade para alcançar um público mais amplo e explorar mercados de plataforma menos saturados.

---
## 3. Gráficos Solicitados pela Fun Corp.

Os gráficos a seguir complementam as análises e fornecem visualizações claras dos dados, seguindo o estilo customizado definido.

### Gráfico 1: Percentual de jogos que possuem suporte para cada sistema operacional. Caso um jogo suporte múltiplos sistemas operacionais, contar um “voto” para cada sistema operacional suportado.

In [ ]:
# Preparar os dados para o Gráfico 1
# É necessário contar a ocorrência de suporte para cada SO
total_games = len(df_steam)
if total_games > 0:
    os_support_counts = {
        'Windows': df_steam['windows'].sum(),
        'Mac': df_steam['mac'].sum(),
        'Linux': df_steam['linux'].sum(),
    }
    os_percentages = {os: (count / total_games) * 100 for os, count in os_support_counts.items()}
    
    print("Percentuais de suporte a sistemas operacionais:")
    for os, pct in os_percentages.items():
        print(f"- {os}: {pct:.2f}%")

    # Gerar o gráfico
    chart_generator.generate_g1_os_support_chart(
        os_percentages,
        title_suffix="Dataset Completo",
        filename_suffix="full_os_support"
    )
    print(f"\nGráfico salvo em '{PLOTS_DIR}/g1_full_os_support.png'")
else:
    print("Não foi possível gerar o gráfico de suporte a SO: DataFrame vazio.")

**Análise Visual:**
O Gráfico 1 ilustra claramente a distribuição do suporte a sistemas operacionais entre os jogos da Steam. É comum observar uma dominância esmagadora do Windows, mas o percentual de jogos com suporte a Mac e Linux também é visível. Essa visualização ajuda a Fun Corp. a entender a fragmentação do mercado de SOs para jogos e a planejar seus esforços de desenvolvimento e portabilidade para atingir diferentes bases de usuários.

### Gráfico 2: Número total de jogos single-player do gênero Indie e estratégia lançados por ano entre 2010 e 2020 (mostrar tendência para cada gênero separadamente, mas no mesmo gráfico).

In [ ]:
# Preparar os dados para o Gráfico 2
# Filtrar jogos single-player (assumindo que 'categories' contém 'Single-player')
# Filtra por 'Indie' e 'Strategy' nos gêneros
# Filtra por ano entre 2010 e 2020

df_filtered_g2 = df_steam[
    (df_steam['release_year'] >= 2010) &
    (df_steam['release_year'] <= 2020) &
    (df_steam['categories'].str.contains('Single-player', case=False, na=False))
].copy()

# Certifica-se de que a coluna 'genres' é string
df_filtered_g2['genres'] = df_filtered_g2['genres'].astype(str)

# Contar jogos Indie e Strategy por ano
indie_by_year = df_filtered_g2[df_filtered_g2['genres'].str.contains('Indie', case=False, na=False)] \
    .groupby('release_year').size().reindex(range(2010, 2021), fill_value=0)

strategy_by_year = df_filtered_g2[df_filtered_g2['genres'].str.contains('Strategy', case=False, na=False)] \
    .groupby('release_year').size().reindex(range(2010, 2021), fill_value=0)

# Criar um DataFrame para o gráfico
genre_trend_df = pd.DataFrame({
    'year': indie_by_year.index,
    'Indie': indie_by_year.values,
    'Strategy': strategy_by_year.values
})

print("Dados de jogos Single-player Indie e Strategy por ano (2010-2020):")
display(genre_trend_df)

# Gerar o gráfico
chart_generator.generate_g2_genre_trend_chart(
    genre_trend_df,
    title_suffix="Dataset Completo",
    filename_suffix="full_indie_strategy_trend"
)
print(f"\nGráfico salvo em '{PLOTS_DIR}/g2_full_indie_strategy_trend.png'")

**Análise Visual:**
O Gráfico 2 apresenta a tendência de lançamento de jogos single-player dos gêneros Indie e Estratégia ao longo dos anos. Esta visualização permite comparar o crescimento ou declínio de cada gênero no período, identificando picos e vales. Pode-se observar se um gênero está em alta ou se estabilizou. Para a Fun Corp., é uma ferramenta valiosa para entender o dinamismo de mercado em gêneros específicos e auxiliar no planejamento estratégico de desenvolvimento de novos títulos.

---
## 4. Pergunta e Gráfico Autoral Adicional

Como analista dedicado, formulei uma pergunta de negócio adicional e criei um gráfico para explorar aspectos mais complexos dos dados.

### Pergunta Autoral: Qual a correlação entre o preço do jogo e o número médio de recomendações (positivas + negativas) para jogos lançados após 2019 em gêneros populares, e como isso varia por gênero?

In [ ]:
price_recs_correlation_by_genre = analyze_price_vs_recommendations_by_genre(df_steam)
print("Correlação entre Preço e Recomendações por Gênero para Jogos Pós-2019:")
display(price_recs_correlation_by_genre)

**Análise:**
Esta análise autoral busca entender se há uma relação entre o preço de um jogo e o nível de engajamento medido por recomendações, focando em títulos mais recentes (pós-2019) e nos gêneros mais populares. Uma correlação positiva indicaria que jogos mais caros em um gênero específico tendem a ter mais recomendações (ou vice-versa), enquanto uma correlação negativa sugeriria o oposto. A ausência de correlação relevante indicaria que o preço e o número de recomendações são fatores independentes. Para a Fun Corp., essa informação é crucial para estratégias de precificação, indicando se um preço premium é justificado pela capacidade de gerar satisfação e buzz entre os jogadores em determinados gêneros.

### Gráfico Autoral: Comparação de Preço Médio e Média de Recomendações por Gênero (Jogos Pós-2019)

In [ ]:
# Gerar o gráfico autoral
chart_generator.generate_g3_custom_chart_price_vs_recs(
    price_recs_correlation_by_genre,
    title_suffix="Dataset Completo",
    filename_suffix="full_price_recs_correlation_by_genre"
)
print(f"\nGráfico salvo em '{PLOTS_DIR}/g3_full_price_recs_correlation_by_genre.png'")

**Análise Visual:**
O gráfico autoral visualiza a relação entre preço médio e recomendações médias por gênero para jogos recentes. Utilizando dois eixos Y, ele permite uma comparação direta de métricas que possuem escalas distintas. Esta visualização facilita a identificação de gêneros onde jogos com preços mais altos ainda conseguem gerar um grande número de recomendações, ou onde a relação é mais balanceada. É uma ferramenta eficaz para validar as descobertas da análise de correlação e oferecer insights estratégicos para o desenvolvimento e marketing de jogos na Fun Corp.

---
## 5. Conclusão

Este projeto da Fase II demonstrou a aplicação prática das bibliotecas `pandas`, `matplotlib` e `numpy` para realizar uma análise de dados abrangente sobre o mercado de jogos da Steam. As respostas às perguntas da Fun Corp. e as visualizações geradas fornecem insights valiosos sobre tendências de mercado, desempenho de gêneros, comportamento de editoras e o impacto do suporte a diferentes sistemas operacionais.

A abordagem modular, com scripts Python separados para processamento de dados, funções de análise e plotagem de gráficos, juntamente com este Notebook Jupyter para orquestração e documentação, otimiza o fluxo de trabalho e a reutilização do código. As análises realizadas e os gráficos apresentados servem como uma base sólida para a Fun Corp. tomar decisões estratégicas mais informadas no dinâmico mercado de jogos digitais.

---